---

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Oracle_logo.svg/2560px-Oracle_logo.svg.png" width="200" align = "left"></p>

# **<h1 align ="middle"><b> Offensive Behavior Lab</b></h1>**

### **<h1 align ="middle"><b> Use case 1. Person Detection in Video</b></h1>**
### **<h1 align ="middle"><b> Use case 2. Offensive Language Detection in Video</b></h1>**

---

#### Welcome to the 2nd notebook. In this notebook, you will perform several steps. Please make sure all previous steps are completed.

## **Prerequisites to run notebook**
- **Note 1**: Perform all steps in the "1_prep_ocw_v1.ipynb" file
- **Note 2**: Pull Github Repository, the repo contains:
    - Three notebooks:
    - A folder named 'job_artifacts', inside the 'job_artifacts' folder:
        - main.py
        - sub_packages with 5 .py files
        - An empty 'wallet' folder
- **Note 3**: A Dynamic Group for OCI Data Science has been created and OCI Data Science policies are in order. Documentation on policies and dynamic group can be found [here](https://docs.oracle.com/en-us/iaas/data-science/using/policies.htm)
- **Note 4**: A correct config file with private key have been generated
- **Note 5**: Only use .jpg images as profile images.
- **Note 6**: YouTube video language should be in English (only for Video analysis)

----

# **Steps in notebook**
---
### !!! Please follow the steps carefully !!!
----


# **Step 1**

### **Config file and Private Key**
- 1. In Lab 1, you generated a **'config'**  and **private_key.pem file.** 
- 2. Upload your 'config' file and your 'private_key.pem' file into the **'job_artifacts'** folder
- 3. You can **'drag & drop'** both files in the directory on the left
----

# **Step 2**

### **Autonomous Database Wallet**
- 1. Go to OCI Homepage, click on **'Oracle Database'** in the menu bar and select **'Autonomous Database'**
- 2. Navigate to the correct compartment and select your autonomous database
- 3. Click on the database name to open the overview page of your autonomous database. Click on **'Database Connection'**
- 4. Click on **Download Wallet** and enter your password. When following this workshop, the password is **"Welcome*12345"**
- 5. In the notebook session, upload the entire wallet in the **"job_artifact/wallet"** folder
- 6. Do not change the name of the wallet. Do not unzip the wallet
----

# **Step 3**

### **Parameters in main.py**
- 1. In **"job_artifacts"** folder, you'll see a file named **"main.py".**
- 2. **Open main.py.** In the first few lines, **ONLY CHANGE THE FOLLOWING 3 VARIABLES**:
    - **password** --> This is password of your user in the Autonomous Database. Example: password = "Welcome*12345"
    - **wallet_name** --> This is the name of the wallet/database, without the 'Wallet_' and without '.zip'. Example: wallet_name = "DB202112101358"  
    - **compartment_id_input** --> This is the Compartment OCID you are working in. Example: compartment_id_input = "ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrjd7wjnw4za3m75q"
- 3. After changing the parameters, save the file (CTRL+S) and close the file.
- 4. See an example of changed parameters in the LiveLab.

----
# **Step 4**

### **Log Group OCID**
- 1. In the OCI Data Science - Job, we need a **Log Group.** In this Log Group, individual Logs will be added on the individual Jobs
- 2. In Lab 1, you created a Log Group.
- 3. Copy the **Log Group OCID** and swap **"ADD_YOUR_LOG_GROUP_OCID_HERE"** with your OCID in the next cell. This ocid will be used in .with_log_group_id() during the creating of the Job. 

----


In [4]:
## Add your Log group OCID below and run this cell by clicking on the 'play' button.
log_group_ocid = "<ADD_YOUR_LOG_GROUP_OCID_HERE>"
print("Your input for Log Group OCID is " + log_group_ocid)

Your input for Log Group OCID is <ADD_YOUR_LOG_GROUP_OCID_HERE>


# **Step 5**

### **Custom Conda Environment**
- 1. In the **'1_prep_ocw_v1.ipynb'** notebook, you created a custom conda environment. That environment is published on Object Storage.
- 2. In the top right corner in the notebook session, make sure to select your custom conda environment, like "Python [YOUR_CUSTOM_CONDA_NAME]]
- 3. Paste the **full location**, including "oci://" in next cell. When following this workshop, your full location is: ***oci://[BUCKET_NAME]@[NAMESPACE]/conda_environments/cpu/offensive_behavior_conda/1.0/offensive_behavior_conda_v1_0***
- 4. Make sure to swap **BUCKET_NAME** and **NAMESPACE** with your bucket name and namespace


In [5]:
# Swap BUCKET_NAME and NAMESPACE and please run this cell
custom_conda_path = "oci://[BUCKET_NAME]@[NAMESPACE]/conda_environments/cpu/offensive_behavior_conda/1.0/offensive_behavior_conda_v1_0"
print("Your input for custom conda is " + custom_conda_path)

Your input for custom conda is oci://[BUCKET_NAME]@[NAMESPACE]/conda_environments/cpu/offensive_behavior_conda/1.0/offensive_behavior_conda_v1_0


---

# **Step 6**

### **Run the notebook below**
#### Run each cell below by clicking on the 'Play' button or step into the cell and hit shift+enter. When starting the Job Run, this will take several minutes. The output of the Job will be refreshed in the notebook.

---

# **| 1. Import libraries**

In [5]:
#imports
from ads.common.oci_logging import OCILogGroup, OCILog
from ads.jobs import Job, DataScienceJob, PythonRuntime
from datetime import datetime, timedelta
from ads import set_auth

#authentication
from ads import set_auth
set_auth(auth='resource_principal')

---

# **| 2. Create an OCI Data Science - Job**

In [12]:
## the below script creates an OCI Data Science - Job

job = (
    Job(name="ocw_my_first_job")
    .with_infrastructure(
        DataScienceJob()
        # Configure logging for getting the job run outputs.
        .with_log_group_id(log_group_ocid)                                                                                                 ###### Add here your Log Group OCID
        .with_shape_name("VM.Standard2.8")                                                                                                 ###### Optional. Change shape to your liking.
        #.with_shape_config_details(memory_in_gbs=16, ocpus=5)
        .with_block_storage_size(50)
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda(custom_conda_path)                                                                                               ###### Add here your Custom Conda Location
        .with_source("/home/datascience/offensive_behavior_conda/job_artifacts/") 
        .with_entrypoint("main.py")
        .with_working_dir("job_artifacts")
        .with_python_path("sub_packages")
        .with_environment_variable(TYPE_OF_ANALYSIS="both",   
                                  YOUTUBE_URL = "no_url_provided",
                                  MAIN_BUCKET_NAME = 'no_bucket_name', 
                                  NAMESPACE_NAME = 'no_name_space_name', 
                                  SCHEMA_NAME = 'no_schema_name'  
                                  )         
    )
)

job.create()


kind: job
spec:
  id: ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaac7kok2yasxo4bgqob2r47rrx7tamzs4xnka6msl5amyayocbhvwq
  infrastructure:
    kind: infrastructure
    spec:
      blockStorageSize: 50
      compartmentId: ocid1.compartment.oc1..aaaaaaaaicxnkb6iumnvekw3xugguxoznng54itwzy5ohvv2x2nczttvusla
      displayName: ocw_my_first_job
      jobInfrastructureType: ME_STANDALONE
      jobType: DEFAULT
      logGroupId: ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaac7kok2yaclpbacc25gy4ys65kmx2ibxcnxsaymmyq73ktlpx5psq
      projectId: ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaac7kok2ya2xks4x6gqchbt24fekj5d3yhii52o2ws5p34e4xnlywa
      shapeName: VM.Standard2.8
    type: dataScienceJob
  name: ocw_my_first_job
  runtime:
    kind: runtime
    spec:
      conda:
        type: published
        uri: oci://ocw_root@oractdemeabdmanalytics/conda_environments/cpu/offensive_behavior_conda/1.0/offensive_behavior_conda_v1_0
      entrypoint: main.py
      env:
      - name: TYPE_OF_ANAL

---

# **| 3. Start the OCI Data Science - Job**

##### Before you start a Job, make sure to add or change the below values.

### **Input Features are:**

| **Variable** | **Values**|
|--|-- | 
|TYPE_OF_ANALYSIS | Use "audio_only"|
|YOUTUBE_URL |The full YouTube URL|
|MAIN_BUCKET_NAME|Bucket name you created in Lab 1|
|NAMESPACE_NAME|Namespace of your tenancy. You can find the namespace in Object Storage|
|SCHEMA_NAME|This is the user name / schema name of the database. Will also be used to create subbuckets in the main bucket|

#### Example YouTube videos of Jimmy Carr:
- https://www.youtube.com/shorts/BsDFzV-Hpsw
- https://www.youtube.com/embed/epU3ipIBSqA?autoplay=1&mute=1    

---

## **Enter your inputs here and run the cell:**

- **Only change the below cell:** add your user name and add your YouTube video you want to analyze. You can use the default YouTube video.
- **Schema_name**. When following this workshop, your user name is "apex_user_1" 

In [ ]:
schema_name = "apex_user_1" 
youtube_url = "<A_YOUTUBE_URL>"
main_bucket_name = "<YOUR_BUCKET_NAME>"
namespace_name = "<YOUR_NAMESPACE>"

## **Start Job Run**

In [14]:
job_run_env = job.run(
    name="Job Run - Passing dynamic values",
    env_var={'TYPE_OF_ANALYSIS': "audio_only",  
                'YOUTUBE_URL': youtube_url, 
                'MAIN_BUCKET_NAME': main_bucket_name,
                'NAMESPACE_NAME': namespace_name,
                'SCHEMA_NAME': schema_name}
)

job_run_watch = job_run_env.watch()

Job OCID: ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaac7kok2yasxo4bgqob2r47rrx7tamzs4xnka6msl5amyayocbhvwq
Job Run OCID: ocid1.datasciencejobrun.oc1.eu-frankfurt-1.amaaaaaac7kok2yasfuqxnk6bb3642ybhwlliwbooo5zosgrq5ivr7z75hha
2023-08-29 09:24:57 - Job Run ACCEPTED
2023-08-29 09:25:09 - Job Run ACCEPTED, Infrastructure provisioning.
2023-08-29 09:26:14 - Job Run ACCEPTED, Infrastructure provisioned.
2023-08-29 09:26:45 - Job Run ACCEPTED, Job run bootstrap starting.
2023-08-29 09:29:37 - Job Run ACCEPTED, Job run bootstrap complete. Artifact execution starting.
2023-08-29 09:29:40 - Job Run IN_PROGRESS, Job run artifact execution in progress.
2023-08-29 09:29:40 - 2023-08-29 09:29:40.188777: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 09:29:40 - 2023-08-29 09:29:40.281289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 09:29:40 - To enable

----

 ## **| (Optional) 4. Query the Autonomous Database to see results**

##### When you would like to see the results in the notebook session, you can query the results here. Add your credentials below and run the cell.

In [ ]:
user_name = "apex_user_1"                          ## User name of the database, same as schema name. Example "ocw"
password = "Welcome*12345"                       ## Password. Example "Welcome*12345"
service_name = "<YOUR_SERVICE_NAME>"             ## The service name is the database name (excluding .zip or Wallet_), including _high, _medium, or _low. Example: db202112101358_high. Make sure you use lower case letters. Example = "db202112101358_high"
full_wallet_name = "<FULL_WALLET_NAME>"          ## Please define your full wallet name, including "Wallet_" and including ".zip". Example: "Wallet_DB202112101358.zip"

In [ ]:
import pandas as pd
import ads

creds = {
    "user_name": user_name,
    "password":  password,
    "service_name": service_name,
    "wallet_location": f"/home/datascience/offensive_behavior_conda/job_artifacts/wallet/{full_wallet_name}"            
}


# simple read of a SQL query into a dataframe with no bind variables
df = pd.DataFrame.ads.read_sql("SELECT * FROM ocw_run_results",connection_parameters=creds)
df.head()

# **The End**